In [2]:
import pandas as pd
import numpy as np
from datapreprocessfunction_code import function_process, readfile, rename_cols,annon,outers_join,add_column,label_paths


In [14]:
%store -r neg
%store -r lncRNAs_ref
%store -r PC_ref

In [5]:
path_lncRNAs = '/home/alva/Documents/lncRNAs_project_2020/Analysis_2021/DNA_methy_tcga/meth/de/filtered_results/'
#reference    = '/'

In [6]:
lncRNAs_common     = readfile(path_lncRNAs,"commonDE_5cancermetainformation_lncRNAs_alone")
lncRNAs_common_ann = annon(lncRNAs_common,"gene_sym",lncRNAs_ref[["chr","start","end","gene_sym",'gene_id']])
cols               = ['chr','start','end','gene_id']
great_results      = readfile(path_lncRNAs,"20210315-public-4.0.4-H80DU8-hg38-all-region.txt")

In [7]:
great_results.head()

,gene_id,PC
0,ENSG00000268895,"A1BG (+1912), ZSCAN22 (+24568)"
1,ENSG00000245105,"A2M (+49231), KLRG1 (+77385)"
2,ENSG00000242908,"SUCNR1 (-33834), AADAC (+25772)"
3,ENSG00000281376,"BCL2L1 (+440), COX4I2 (+84570)"
4,ENSG00000231749,"ABCA8 (-38417), ABCA9 (+67406)"


In [8]:
def Cis_Trans1(df):
    df1         = df.PC.str.split(',', expand=True)
    regex       = r'(?P<PC>\w+.*).*\((?P<Strand>[+-])(?P<Distance>.*)\)'
    extract     = lambda df: df.iloc[0].str.extract(regex, expand=True)
    extracted   = df1.groupby(level=0).apply(extract)
    df2         = extracted.stack().unstack([2, 1])
    colseries   = df2.columns.to_series()
    df2.columns = colseries.str.get(0).astype(str) + colseries.str.get(1).astype(str)
    PCs         = pd.concat([df, df2], axis=1)
    return(PCs)

In [9]:
def arrange_cols(df):
    frames_to_concat = []
    for col in df:
        if col.startswith("Dis"):
            dis_num = col[-1] # Extract the # from a column like Distance# or Dis#
            frame_to_concat = df[["lncRNA", "PC" + dis_num, "Distance" + dis_num]]
            frame_to_concat.columns = ["lncRNA", "PC", "Distance"]
            frames_to_concat.append(frame_to_concat)
    stacked_columns = pd.concat(frames_to_concat)
    stacked_columns = stacked_columns[stacked_columns["Distance"] != "NaN"]
    stacked_columns = stacked_columns[stacked_columns["Distance"] != "None"]
    return stacked_columns


In [10]:
great = Cis_Trans1(great_results)
great = rename_cols(great,"gene_id","lncRNA")

In [13]:
arr_great       = arrange_cols(great)
#arr_great_less = arr_great.query('Distance <= "400000"')


Stored 'arr_great' (DataFrame)


In [15]:
arr_great     = rename_cols(arr_great,"PC","gene_sym")
arr_great['gene_sym'] = arr_great['gene_sym'].str.strip()
arr_great_ann = annon(arr_great,"gene_sym",PC_ref[["gene_sym","gene_id"]])
%store arr_great
#arr_great_ann.to_csv(path_lncRNAs+'great_PC_commonDE5lncRNAs', sep='\t',index=False)

Stored 'arr_great' (DataFrame)


In [17]:
lncRNAs_common_ann[cols].to_csv(path_lncRNAs+'commonDE_5cancermetainformation_lncRNAs_alone_chr', sep='\t',index=False)

In [1]:
%store lncRNAs_common_ann

UsageError: Unknown variable 'lncRNAs_common_ann'
